### Ques 1

In [5]:
!pip install gurobipy

### Formulation

 **Variables**:
- $( y_j )$: Number of times combination j  is used (non-negative integer).
- $( x_{j,i} )$: Amount of resource i  used by combination  j.
- $( t_i )$: Total amount of resource  i  used.

**Objective**:
Minimize the total number of rolls used:

$ \min \left( \sum_{j} y_j \right)$

**Constraints**:
1. Demand for each resource i must be satisfied:

$ \sum_{j} x_{j,i} \cdot y_j \geq b_i \quad \forall i $

2. Resource usage for each resource i :

$ t_i = \sum_{j} x_{j,i} \cdot y_j \quad \forall i $

3. $( y_j \geq 0 )$ and integer for all \( j \).


In [6]:
from pyomo.environ import *

L = 1030
a = [1000, 980, 940, 640, 630, 60, 55, 50, 45, 40]
b = [76, 50, 1189, 3876, 4187, 945, 817, 138, 1380, 140]

def pt(a, L):
    x = []
    def pt1(p, r, k):
        if k == len(a):
            x.append(p[:])
            return
        pt1(p, r, k + 1)
        if a[k] <= r:
            p[k] += 1
            pt1(p, r - a[k], k)
            p[k] -= 1
    pt1([0] * len(a), L, 0)
    return x

x = pt(a, L)

In [7]:
def solve(x):
    m = ConcreteModel()
    m.y = Var(range(len(x)), within=NonNegativeIntegers)
    m.z = Objective(expr=sum(m.y[j] for j in range(len(x))), sense=minimize)

    def c(m, i):
        return sum(x[j][i] * m.y[j] for j in range(len(x))) >= b[i]

    m.d = ConstraintList()
    for i in range(len(a)):
        m.d.add(c(m, i))

    solver = SolverFactory('gurobi')
    result = solver.solve(m)

    return m, m.z()

In [8]:
def display(x, m, z):
    print(f"Minimum number of used rolls: {z:.0f}")
    for j in range(len(x)):
        if m.y[j].value > 0:
            print(f"Combination {x[j]} used {m.y[j].value:.0f} times")
    
    t = [0] * len(a)
    for i in range(len(a)):
        for j in range(len(x)):
            if m.y[j].value > 0:
                t[i] += x[j][i] * m.y[j].value
    print(f"Total resource usage: {t}")

In [9]:
m, z = solve(x)
display(x, m, z)

Minimum number of used rolls: 9378
Combination [0, 0, 0, 0, 1, 0, 6, 0, 1, 0] used 1380 times
Combination [0, 0, 0, 0, 1, 0, 6, 1, 0, 0] used 2807 times
Combination [0, 0, 0, 1, 0, 1, 6, 0, 0, 0] used 3876 times
Combination [0, 0, 1, 0, 0, 0, 0, 0, 0, 2] used 1189 times
Combination [0, 1, 0, 0, 0, 0, 0, 0, 0, 1] used 50 times
Combination [1, 0, 0, 0, 0, 0, 0, 0, 0, 0] used 76 times
Total resource usage: [76.0, 50.0, 1189.0, 3876.0, 4187.0, 3876.0, 48378.0, 2807.0, 1380.0, 2428.0]


In [10]:
L1 = 1030
a1 = [1000, 980, 940, 920, 900, 710, 700, 650, 640, 630, 60, 55, 50, 45, 40]
b = [76, 50, 1189, 2210, 987, 894, 2850, 6603, 3876, 4187, 945, 817, 138, 1380, 140]

x1 = pt(a1, L1)


In [11]:
m, z = solve(x1)
display(x1, m, z)

Minimum number of used rolls: 22922
Combination [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0] used 4187 times
Combination [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0] used 3876 times
Combination [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0] used 6603 times
Combination [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0] used 2850 times
Combination [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0] used 894 times
Combination [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] used 987 times
Combination [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] used 2210 times
Combination [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] used 1189 times
Combination [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] used 50 times
Combination [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] used 76 times
Total resource usage: [76.0, 50.0, 1189.0, 2210.0, 987.0, 894.0, 2850.0, 6603.0, 3876.0, 4187.0]
